In [1]:
import json
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
from selenium import webdriver as wb
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
dong = ['상무1동', '상무2동', '금호1동', '금호2동', '화정1동', '화정2동', '화정3동', '화정4동', '양동', '양3동', '농성1동', '농성2동', '광천동', '유덕동', '치평동', '서창동', '동천동', '풍암동']

dong1 = ['상무1동', '상무2동', '금호1동', '금호2동'] # 남규형
dong2 = ['화정1동', '화정2동', '화정3동', '화정4동'] # 진우형
dong3 = ['양동', '양3동', '농성1동', '농성2동'] # 승현이
dong4 = ['광천동', '유덕동', '치평동', '서창동', '동천동', '풍암동']

bigcode = ["G2", "I2", "S2", "R1", "P1", "L1", "I1", "M1", "Q1", "N1"]

In [3]:
category_df = pd.read_csv("업종.csv", encoding='utf-8-sig')

In [4]:
columns = ['구', '업종코드', '업종이름', '검색업종','분석 영역','매출', '23.12', '24.01', '24.02', '24.03', '24.04', 
           '24.05', '24.06', '24.07', '24.08', '24.09', '24.10', '24.11', '24.12']
columns_ = ['구', '업종코드', '업종이름', '검색업종', '23.12', '24.01', '24.02', '24.03', '24.04', 
           '24.05', '24.06', '24.07', '24.08', '24.09', '24.10', '24.11', '24.12']
delivery_sales_df  = pd.DataFrame(columns=columns_)
store_sales_df  = pd.DataFrame(columns=columns)

In [5]:
s = Service(ChromeDriverManager().install())
driver = wb.Chrome(service=s)
driver.maximize_window()
driver.get('https://bigdata.sbiz.or.kr/gis/?type=detail&rptpType=gisDetail&lat=35.1287735&lng=126.8532396')

In [6]:
error_cate = []
old_dong = ''

In [13]:
#################################### 반복할 동 선택
seogu = dong2[:]                   # 만약 두번째 동을 시작해야 할 경우 dong[1:] 와 같이 인덱스 입력
####################################
turn = 175                           # 중간에 에러가 났을 경우
#################################### 프린트 되어있는 진행횟수에 적혀있는 숫자 입력

In [14]:
try:
    for dong in seogu: #### 구 반복 ####
        flag = True
        
        if old_dong != dong: # 동이 바뀌었을 경우
            turn = 0 # 진행도 초기화

        while flag:
            time.sleep(1)
            search_box = driver.find_element(By.ID, "searchAddress")  # 검색창

            time.sleep(1)
            search_box.send_keys(Keys.CONTROL + "a")  # 동 입력창 초기화
            time.sleep(1)
            search_box.send_keys(Keys.DELETE)
            time.sleep(1)

            search_box.send_keys(f"광주 서구 {dong}")  # 입력창에 원하는 동 입력
            search_box.send_keys(Keys.RETURN)
            time.sleep(1)

            button = driver.find_element(By.CSS_SELECTOR, "ul#admListAddress>li>button") # 검색
            
            if button.text == f"광주 서구 {dong}":
                flag = False                              # while 나가기 -> 동 검색 끝
                button.click()
            else:
                print("동 찾기 에러")
                time.sleep(3)

        for i in range(turn, len(category_df)):           #### 세부업종 구간 ####
            time.sleep(1)
            Upjong = driver.find_element(By.ID, "selectedUpjong") # 업종 입력창

            Upjong.send_keys(Keys.CONTROL + "a")  # 업종 입력창 초기화
            Upjong.send_keys(Keys.DELETE)
            time.sleep(1)

            up_name = category_df["업종이름"].iloc[i] # 입력할 업종 불러오기
            up_code = category_df["업종코드"].iloc[i]

            Upjong.send_keys("> " + up_name)  # 입력창에 업종 검색
            Upjong.send_keys(Keys.RETURN)
            time.sleep(1)

            search = driver.find_elements(By.CSS_SELECTOR, "ul#tpbizSearchListUl>li>button")# 연관 검색 클릭
            for j in range(len(search)):
                if search[j].text.find(up_name) != -1:
                    button = search[j]
                    break
            button.click()
            time.sleep(1)

            button = driver.find_element(By.ID, "analysisBtn")  # 분석하기 버튼 클릭
            button.click()
            time.sleep(8)

            td_name= driver.find_elements(By.CSS_SELECTOR,'div.boxArea>div.summary>dl') # 상세 페이지에 있는 업종 명 가져오기
            for k in range(len(td_name)):                                               #    (제대로 가져왔는지 확인을 위해)
                if td_name[k].text.find(up_name) != -1:
                    td_name = td_name[k].text

            # 상가 매출 데이터 가져오기
            tr = driver.find_elements(By.CSS_SELECTOR, "div.boxArea.type4.space>div.tableArea>div.scrollX>table>tbody>tr") # 상세 보고서 테이블들
            try:
                for td in tr: # 매출 테이블 찾기
                    if td.text.find('분석영역 매출') != -1:
                        store_sales_data = td.text.split(" ") # 상가 매출 데이터 추가작업
                        store_sales_df.loc[i] = [dong, up_code, up_name, td_name] + store_sales_data # 최종 데이터 DF에 추가
                        break
                    elif td.text.find(f'{dong} 매출') != -1:
                        store_sales_data = td.text.split(" ") # 상가 매출 데이터 추가작업
                        store_sales_df.loc[i] = [dong, up_code, up_name, td_name] + store_sales_data # 최종 데이터 DF에 추가
                        break
                    elif td.text.find(f'서구 매출') != -1:
                        store_sales_data = td.text.split(" ") # 상가 매출 데이터 추가작업
                        store_sales_df.loc[i] = [dong, up_code, up_name, td_name] + store_sales_data # 최종 데이터 DF에 추가
                        break
                    
            except:
                error_cate.append((td_name, turn)) # 매출 테이블을 찾지 못했을 경우

            # 배달 매출 데이터 가져오기
            try :                                                            # 배달 매출 txt 바로 리스트로 바꿔 넣기
                delivery_sales_data = driver.find_element(By.CSS_SELECTOR, "tr#genderAllSum").text.split(" ")[2:]
                delivery_sales_df.loc[i] = [dong, up_code, up_name, td_name] + delivery_sales_data # 최종 데이터 DF에 추가
            except:
                pass # 배달 매출 정보가 없을 경우 배달을 하지 않는 업종이므로 pass

            # 상세 분석 나가기
            try:
                close = driver.find_element(By.CSS_SELECTOR, "section#reportDiv>div.reportInner>button.close")
                close.click()
            except:
                time.sleep(3)
                close = driver.find_element(By.CSS_SELECTOR, "section#reportDiv>div.reportInner>button.close")
                close.click()
                
            turn = i # 진행도 저장
            old_dong = dong # 진행하고 있는 동 저장
            print(f"{dong}, 진행 횟수: {turn + 1}, 남은 횟수: {len(category_df)-(turn+1)}")

        ## 저장 후 DF초기화
        store_sales_df.to_csv(f"./data/01_상가/{dong}_상가_매출.csv", encoding='utf-8-sig', index=False)
        delivery_sales_df.to_csv(f"./data/02_배달/{dong}_배달_매출.csv", encoding='utf-8-sig', index=False)
        store_sales_df = pd.DataFrame(columns=columns)
        delivery_sales_df = pd.DataFrame(columns=columns_)
        
except Exception as e:
    print("에러 발생:", e)

화정1동, 진행 횟수: 176, 남은 횟수: 71
화정1동, 진행 횟수: 177, 남은 횟수: 70
화정1동, 진행 횟수: 178, 남은 횟수: 69
화정1동, 진행 횟수: 179, 남은 횟수: 68
화정1동, 진행 횟수: 180, 남은 횟수: 67
화정1동, 진행 횟수: 181, 남은 횟수: 66
화정1동, 진행 횟수: 182, 남은 횟수: 65
화정1동, 진행 횟수: 183, 남은 횟수: 64
화정1동, 진행 횟수: 184, 남은 횟수: 63
화정1동, 진행 횟수: 185, 남은 횟수: 62
화정1동, 진행 횟수: 186, 남은 횟수: 61
화정1동, 진행 횟수: 187, 남은 횟수: 60
화정1동, 진행 횟수: 188, 남은 횟수: 59
화정1동, 진행 횟수: 189, 남은 횟수: 58
화정1동, 진행 횟수: 190, 남은 횟수: 57
화정1동, 진행 횟수: 191, 남은 횟수: 56
화정1동, 진행 횟수: 192, 남은 횟수: 55
화정1동, 진행 횟수: 193, 남은 횟수: 54
화정1동, 진행 횟수: 194, 남은 횟수: 53
화정1동, 진행 횟수: 195, 남은 횟수: 52
화정1동, 진행 횟수: 196, 남은 횟수: 51
화정1동, 진행 횟수: 197, 남은 횟수: 50
화정1동, 진행 횟수: 198, 남은 횟수: 49
화정1동, 진행 횟수: 199, 남은 횟수: 48
화정1동, 진행 횟수: 200, 남은 횟수: 47
화정1동, 진행 횟수: 201, 남은 횟수: 46
화정1동, 진행 횟수: 202, 남은 횟수: 45
화정1동, 진행 횟수: 203, 남은 횟수: 44
화정1동, 진행 횟수: 204, 남은 횟수: 43
화정1동, 진행 횟수: 205, 남은 횟수: 42
화정1동, 진행 횟수: 206, 남은 횟수: 41
화정1동, 진행 횟수: 207, 남은 횟수: 40
화정1동, 진행 횟수: 208, 남은 횟수: 39
화정1동, 진행 횟수: 209, 남은 횟수: 38
화정1동, 진행 횟수: 210, 남은 횟수: 37
화정1동, 진행 횟수: 211, 남은

In [9]:
print("상가 매출을찾지 못한 부분() => 따로 확인해주세용\n{}".format(error_cate))

상가 매출을찾지 못한 부분() => 따로 확인해주세용
[]


In [10]:
for i in range(len(error_cate)):
    print((error_cate[i][1]))
    # for j in range(len(error_cate[0][i])):
    #     print(error_cate[0][i][j])
print(category_df.iloc[40])
# error_cate[0][0][0].text

업종이름        서점
업종코드    G21301
Name: 40, dtype: object


API를 이용해서 상가 분류를 중->소분류로 구분

In [11]:
# Key = ''
# small_name = []
# small_code = []

# for i in bigcode:
#     middleUpjongList = f'http://apis.data.go.kr/B553077/api/open/sdsc2/middleUpjongList?serviceKey={Key}&indsLclsCd={i}&type=json'
#     middle_json = requests.get(middleUpjongList)
#     json_ob_1 = json.loads(middle_json.text)
#     middle_body = json_ob_1['body']['items']

#     for j in middle_body:
#         smallUpjongList = f'http://apis.data.go.kr/B553077/api/open/sdsc2/smallUpjongList?serviceKey={Key}&indsLclsCd={i}&indsMclsCd={j.get('indsMclsCd')}&type=json'
#         json_ob_2 = json.loads(requests.get(smallUpjongList).text)
#         small_body = json_ob_2['body']['items']
#         for k in small_body:
#             small_name.append(k.get('indsSclsNm'))
#             small_code.append(k.get('indsSclsCd'))
            
# print("업종코드 갯수 : {}".format(len(small_code)))

df를 불러와서 중복이 있는지 확인

In [12]:
# df = pd.read_csv("농성1동_상가_매출.csv", encoding='utf-8-sig')
# err = []
# for i in range(len(df)):
#     if df.loc[i, '업종이름'] != df.loc[i, '검색업종'].split('> ')[-1]:
#         print(df.loc[i, '업종이름'])
#         err.append(df.loc[i, '업종이름'])

트러블 슈팅

소분류 업종을 입력했을 때 중복된 이름을 가진 다른 업종을 선택하게 되는 경우가 있었다 -> "업종이름" 에서 "> 업종이름" 으로 변경해서 수정



분석 영역에서 매출 데이터를 가져왔지만, 불규칙적으로 동네 이름이 적혀있거나, 동의 매출도 없어서 서구 데이터만 가져오는 경우가 생김
-> 분석영역 매출 and 동 매출로 변경